In [ ]:
# Loading libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
from random import randrange, uniform

os.chdir("C:/Users/jatin/Desktop/python_edwisor")

In [ ]:
os.getcwd()

In [ ]:
df = pd.read_csv("Train_data.csv")
df.shape

# We have 3333 observations of 21 variables

In [ ]:
# Saving a copy of df before applying data pre -processing steps
df_one = df


In [ ]:
# After the data is loaded , I have to check for missing values as pre- processing step
missing_val = pd.DataFrame(df.isnull().sum())
missing_val

# there are no mising values we have to go for outlier analysis now

In [ ]:
%matplotlib inline
plt.boxplot(df['number customer service calls'])

#df.columns


In [ ]:
# Save numeric data
cnames = ['account length',  'number vmail messages',
       'total day minutes', 'total day calls', 'total day charge',
       'total eve minutes', 'total eve calls', 'total eve charge',
       'total night minutes', 'total night calls', 'total night charge',
       'total intl minutes', 'total intl calls', 'total intl charge',
       'number customer service calls']
cnames

In [ ]:
for i in cnames:
    print(i)
    q75, q25 = np.percentile(df.loc[:,i], [75,25])
    iqr = q75-q25
    min = q25 - 1.5*iqr
    max = q75 + 1.5*iqr
    df = df.drop(df[df.loc[:,i]<min].index)
    df = df.drop(df[df.loc[:,i]>max].index)

In [ ]:
df.shape

# Now after removing ouliers we have 2797 observations of 21 variables
# Next step is feature selection 

In [ ]:
# We will corelation analysis among independent variables to remove redudant variables

df_corr = df.loc[:,cnames]
df_corr.shape

f, ax = plt.subplots(figsize=(7, 5))



In [ ]:
corr = df_corr.corr()
corr

In [ ]:
!pip install seaborn

import seaborn as sns
import matplotlib 

%matplotlib inline
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)


# We have corelation value of 1 for::
#total day minutes and total day charge
#total night minutes and total night charge
#total evening minutes and total evening charge
#total intl minutes and total int charges

#Hence we will remove total day charge , total night charge,total evening charge and total intl charge


In [ ]:
df.columns


In [ ]:
cat_names = ['state',"area code","phone number","international plan" ,"voice mail plan"]
cat_names


In [ ]:
for i in cat_names:
    print(i)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df['Churn'], df[i]))
    print(p)
    
    # p value is greater than 0.05 for area code and phone number .Hence we will remove them 
    # Also state value is 0.02 which is not very significiant and also it contains 51 levels, will remove this as it will not contribute
    # for model building

In [ ]:
# We can eliminate state , area code , phone no . and other numeric variables

In [ ]:
df = df.drop(["state","area code", "phone number","total day charge" ,"total eve charge", "total night charge" , "total intl charge"] , axis =1) 

In [ ]:
df
df.shape

#we have removed 7 varibales now ...14 variables with 2797 observations

In [ ]:
cnames = ['account length',  'number vmail messages',
       'total day minutes', 'total day calls', 
       'total eve minutes', 'total eve calls',
       'total night minutes', 'total night calls', 
       'total intl minutes', 'total intl calls', 
       'number customer service calls']
cnames

#Feature scaling..like if we go for knn
#Nomalisation
df_featured = df.copy()
for i in cnames:
    print(i)
    df_featured.loc[:,i] = (df.loc[:,i] - np.min(df[i]))/(np.max(df[i]) - np.min(df[i]))
df_featured

In [ ]:
#Import Libraries for decision tree
from sklearn import tree
from sklearn.metrics import accuracy_score

In [ ]:

# Converting strings to numeric codes for model building for all independent variables
for i in range(0, (df.shape[1]-1)):
    print(i)
    if(df.iloc[:,i].dtypes == 'object'):
        df.iloc[:,i] = pd.Categorical(df.iloc[:,i])
        #print(marketing_train[[i]])
        df.iloc[:,i] = df.iloc[:,i].cat.codes 
        df.iloc[:,i] = df.iloc[:,i].astype('object')
df


In [ ]:

# Creating train data for x and y
x_train = df.values[:, 0:13]
x_train
y_train = df.values[:,13]
y_train


In [ ]:
#Decision Tree
C50_model = tree.DecisionTreeClassifier(criterion='entropy').fit(x_train, y_train)
C50_model



In [ ]:
# Reading test data from Testdata.csv


In [ ]:
test = pd.read_csv("Test_data.csv")
test

In [ ]:
# Remove redundant variables from test data as in train data

test = test.drop(["state","area code","phone number","total day charge","total night charge", "total eve charge","total intl charge"] , axis =1)




In [ ]:
# convert string values of test data frame to numeric values
for i in range(0, (test.shape[1]-1)):
    print(i)
    if(test.iloc[:,i].dtypes == 'object'):
        test.iloc[:,i] = pd.Categorical(test.iloc[:,i])
        #print(marketing_train[[i]])
        test.iloc[:,i] = test.iloc[:,i].cat.codes 
        test.iloc[:,i] = test.iloc[:,i].astype('object')
test

In [ ]:

# Creating test values for x and y 
x_test = test.values[:,0:13]
y_test = test.values[:,13]
print(y_test)

In [ ]:
C50_Predictions = C50_model.predict(x_test)
C50_Predictions


In [ ]:
from sklearn.metrics import confusion_matrix 
CM = confusion_matrix(y_test,C50_Predictions)
CM
CM = pd.crosstab(y_test, C50_Predictions)
CM




In [ ]:
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

Accuracy =((TP+TN)*100)/(TP+TN+FP+FN)
Accuracy

In [ ]:
FNR = (FN*100)/(FN+TP)
FNR

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF_model = RandomForestClassifier(n_estimators = 50).fit(x_train, y_train)

In [ ]:
RF_Predictions = RF_model.predict(x_test)

In [ ]:
CM = pd.crosstab(y_test, RF_Predictions)
CM

In [ ]:
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]


In [ ]:
Accuracy = ((TP+TN)*100)/(TP+TN+FP+FN)
Accuracy
 

In [ ]:
FNR = (FN*100)/(FN+TP)
FNR

In [ ]:
 df = df.reset_index(drop=True)


#Let us prepare data for logistic regression
#replace target categories with 0 AND 1s
for i in range(0,len(df)):
    df.loc[i,'Churn'] = df.loc[i,'Churn'].replace('False', "0")
    df.loc[i,'Churn'] = df.loc[i,'Churn'].replace('True', "1")
df


In [ ]:
df

In [ ]:
#Create logistic data. Save target variable first
df_logit = pd.DataFrame(df['Churn'])

In [ ]:
df_logit['Churn'].dtype

In [ ]:
cnames
print(cnames)
df_logit
#Add continous variables
df_logit = df_logit.join(df[cnames])
df_logit

In [ ]:
# Now adding categorical variables with dummy encoding

##Create dummies for categorical variables
cat_names = ["international plan" , "voice mail plan"]

for i in cat_names:
    temp = pd.get_dummies(df[i], prefix = i)
    df_logit = df_logit.join(temp)


In [ ]:
df_logit

In [ ]:
# Now we have training set with dummy variables of categories


In [ ]:
#select column indexes for independent variables
train_cols = df_logit.columns[1:16]
train_cols


In [ ]:

np.asarray(df_logit[train_cols]).dtype
 

In [ ]:
df_logit["Churn"] = pd.to_numeric(df_logit["Churn"])
df_logit['Churn'].dtype

In [ ]:
df_logit

In [ ]:
!pip install statsmodels

#Built Logistic Regression
import statsmodels.api as sm

logit = sm.Logit(df_logit['Churn'], df_logit[train_cols]).fit()

logit.summary()


In [ ]:
 #Now preparing test data 
for i in range(0,len(test)):   
    test.loc[i,'Churn'] = test.loc[i,'Churn'].replace('False', "0")
    test.loc[i,'Churn'] = test.loc[i,'Churn'].replace('True', "1")
test

In [ ]:
test_logit = pd.DataFrame(test['Churn'])

In [ ]:
test_logit['Churn'].dtype

In [ ]:
test_logit["Churn"]= pd.to_numeric(test_logit["Churn"])

In [ ]:
test_logit = test_logit.join(test[cnames])


In [ ]:
for i in cat_names:
    temp = pd.get_dummies(test[i],prefix=i)
    test_logit = test_logit.join(temp)
    

In [ ]:
test_logit

In [ ]:
#Predict test data
test_logit['Actual_prob'] = logit.predict(test_logit[train_cols])

test_logit['ActualVal'] = 1
test_logit.loc[test_logit.Actual_prob < 0.5, 'ActualVal'] = 0

In [ ]:
test_logit

In [ ]:
#Build confusion matrix
CM = pd.crosstab(test_logit['Churn'], test_logit['ActualVal'])

#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model
#accuracy_score(y_test, y_pred)*100
A= (TP+TN)*100/(TP+TN+FP+FN)
FNR =(FN*100)/(FN+TP)

In [ ]:
FNR

In [ ]:
A

In [ ]:
# accuracy = 89.02 , false = 73.21 for logistic

In [ ]:
df_featured

In [ ]:
cnames_new = {"international plan","voice mail plan","Churn"}
cnames_new

In [ ]:
for i in range(0, df_featured.shape[1]):
    #print(i)
    if(df_featured.iloc[:,i].dtypes == 'object'):
        df_featured.iloc[:,i] = pd.Categorical(df_featured.iloc[:,i])
        #print(marketing_train[[i]])
        df_featured.iloc[:,i] = df_featured.iloc[:,i].cat.codes 
     

In [ ]:
df_featured

In [ ]:
x_train = df_featured.iloc[:,0:13]
x_train

In [ ]:
y_train = df_featured.iloc[:,13]
y_train

In [ ]:
# For KNN we will implement df_featured (scaled data)
#KNN implementation
from sklearn.neighbors import KNeighborsClassifier

KNN_model = KNeighborsClassifier(n_neighbors = 5).fit(x_train, y_train)


In [ ]:
test

In [ ]:
test_featured = test.copy()
for i in cnames:
    print(i)
    test_featured.loc[:,i] = (test.loc[:,i] - np.min(test[i]))/(np.max(test[i]) - np.min(test[i]))
test_featured

In [ ]:
x_test = test_featured.iloc[:,0:13]
y_test= test_featured.iloc[:,13]

In [ ]:
KNN_Predictions = KNN_model.predict(x_test)

In [ ]:
CM = pd.crosstab(y_test, KNN_Predictions)


In [ ]:
#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model
#accuracy_score(y_test, y_pred)*100
A = ((TP+TN)*100)/(TP+TN+FP+FN)

#False Negative rate 
FNR= (FN*100)/(FN+TP)

In [ ]:
A

In [ ]:
FNR

In [ ]:
from sklearn.naive_bayes import GaussianNB

#Naive Bayes implementation
NB_model = GaussianNB().fit(df.iloc[:,0:13], df.iloc[:,13])

In [ ]:
NB_Predictions = NB_model.predict(test.iloc[:,0:13])

In [ ]:
CM = pd.crosstab(test.iloc[:,13], NB_Predictions)


In [ ]:
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model
#accuracy_score(y_test, y_pred)*100
A=((TP+TN)*100)/(TP+TN+FP+FN)

#False Negative rate 
FNR=(FN*100)/(FN+TP)


In [ ]:
A

In [ ]:
FNR